In [1]:
import numpy as np
import Ipynb_importer
import homography
from scipy import ndimage
from scipy.spatial import Delaunay
from PIL import Image
import matplotlib.pyplot as plt

importing Jupyter notebook from homography.ipynb
importing Jupyter notebook from warp.ipynb


In [2]:
def image_in_image(im1,im2,tp):
    """使用仿射变换im1放在im2上，使im1图像的角和tp尽可能地接近
    tp是其次表示的，并且是按照从左上角逆时针计算的"""
    
    #扭曲的点
    m,n = im1.shape[:2]
    fp = np.array([[0,m,m,0],[0,0,n,n],[1,1,1,1]])
    #print('fp:\n',fp)
    
    #计算仿射变换并将其应用于图像im1
    #H = homography.Haffine_from_points(fp,tp)
    H = homography.Haffine_from_points(fp,tp)
    H_inv = np.linalg.inv(H)
    im1_t = ndimage.affine_transform(im1,H_inv[:2,:2],(H_inv[0,2],H_inv[1,2]),im2.shape[:2])
    #print(im1_t)
    alpha = (im1_t > 0)
    
    return (1-alpha)*im2 + alpha*im1_t,H

In [3]:
def alpha_for_triangle(points,m,n):
    """对于带有由points定义角点的三角形，创建大小为(m,n)的alpha图"""
    #print('points\n',points,'\n')
    alpha = np.zeros((m,n))
    for i in range(np.min(points[0]),np.max(points[0])):
        for j in range(np.min(points[1]),np.max(points[1])):
            x = np.linalg.solve(points,[i,j,1])
            if(np.min(x) > 0):    #所有系数大于0
                alpha[i,j] = 1
    
    return alpha

In [4]:
def triangulate_points(x,y):
    """二维点的Delaunay三角剖分"""
    
    tri = Delaunay(np.c_[x,y]).simplices
    
    return tri

In [5]:
def pw_affine(fromim,toim,fp,tp,tri):
    """从一幅图像中扭曲举行图像块
    formim = 将要扭曲的图像
    toim = 目标图像
    fp = 齐次坐标下扭曲前的点
    tp = 齐次坐标下扭曲后的点
    tir = 三角剖分"""
    
    im = toim.copy()
    
    #检查图像是灰度图像还是彩色图像
    is_color = len(fromim.shape) == 3
    
    #创建扭曲后的图像（如果需要对彩色图像的每个颜色通道进行迭代操作，那么有必要这样做）
    im_t = np.zeros(im.shape,'uint8')
    
    for t in tri:
        #计算仿射变换
        H = homography.Haffine_from_points(tp[:,t],fp[:,t])
        
        if is_color:
            for col in range(fromim.shape[2]):
                im_t[:,:,col] = ndimage.affine_transform(fromim[:,:,col],H[:2,:2],(H[0,2],H[1,2]),im.shape[:2])
        else:
            im_t = ndimage,affine_transform(fromim,H[:2,:2],(H[0,2],H[1,2]),im.shape[:2])
            
        #三角形的alpha
        #print(tp)
        #print('t=',t,'\n')
        #print(tp[:,t])
        #print(im.shape[0])
        #print(im.shape[1])
        alpha = alpha_for_triangle(tp[:,t],im.shape[0],im.shape[1])
        
        #将三角形加入到图像中
        im[alpha>0] = im_t[alpha>0]
        
    return im

In [ ]:
def plot_mesh(x,y,tri):
    """绘制三角形"""
    
    for t in tri:
        t_ext = [t[0],t[1],t[2],t[0]]    #将第一个点加入最后
        plt.plot(x[t_ext],y[t_ext],'r')

#打开图像，并将其扭曲
fromim = np.array(Image.open('sunset_tree.jpg'))
x,y = np.meshgrid(range(5),range(6))
#print(x)
#print(y)
x = (fromim.shape[1]/4)*x.flatten()
y = (fromim.shape[0]/5)*y.flatten()

#三角剖分
tri = triangulate_points(x,y)

#打开图像和目标点
im = np.array(Image.open('turningtorso1.jpg'))
tp = np.loadtxt('turningtorso1_points.txt',dtype=int)    #destination points
#print(tp)

#将点转换成齐次坐标
fp = np.vstack((y,x,np.ones((1,len(tp)))))
tp = np.vstack((tp[:,1],tp[:,0],np.ones((1,len(tp)))))
tp.dtype = 'int'
#print(tp)

#扭曲三角形
im = pw_affine(fromim,im,fp,tp,tri)

In [ ]:
def im_plot(im):
    plt.figure()
    plt.imshow(im.astype('uint8'))
    plt.show()

In [ ]:
def panorama_change(H,fromim,toim,padding=2400,delta=2400):
    """ 使用单应性矩阵H（使用RANSAC稳健性估计得出），协调两幅图像，创建水平全景图。结果
        为一幅和toim具有相同高度的图像。padding指定填充像素的数目，delta指定额外的平移量""" 
    print(H)
    im_plot(fromim)
    im_plot(toim)
    #imfrom = fromim
    #imto = toim
    
    #检查图像是灰度图像，还是彩色图像
    is_color = len(fromim.shape) == 3
    #print(is_color)
    
    # 用于geometric_transform()的单应性变换
    def transf(p):
        p2 = np.dot(H,[p[0],p[1],1])
        return (p2[0]/p2[2],p2[1]/p2[2])
    
    if H[0,2]>0: # fromim在右边
        print ('warp - right')
        # 变换fromim
        if is_color:
            # 在目标图像的右边填充0
            #im_plot(toim)
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding,3))))
            im_plot(toim_t)
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],transf,(toim.shape[0],toim.shape[1]+padding))
        else:
            # 在目标图像的右边填充0
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding))))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding)) 
    else:
        print ('warp - left')
        #为了补偿填充效果，在左边加入平移量
        H_delta = np.array([[1,0,0],[0,1,-delta],[0,0,1]])
        H = np.dot(H,H_delta)
        #fromim变换
        if is_color:
            # 在目标图像的左边填充0
            toim_t = np.hstack((np.zeros((toim.shape[0],padding,3)),toim))
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            im_plot(fromim_t)
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],transf,(toim.shape[0],toim.shape[1]+padding))
            im_plot(fromim_t)
        else:
            # 在目标图像的左边填充0
            toim_t = np.hstack((np.zeros((toim.shape[0],padding)),toim))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding))
    
    # 协调后返回（将fromim放在toim上）
    im_plot(fromim_t)
    im_plot(toim_t)
    if is_color:
        # 所有非黑像素
        alpha = ((fromim_t[:,:,0] * fromim_t[:,:,1] * fromim_t[:,:,2] ) > 0)
        return fromim_t,toim_t,alpha
        for col in range(3):
            #toim_t[:,:,col] = fromim_t[:,:,col]*alpha + toim_t[:,:,col]*(1-alpha)
            toim_t[:,:,col] = fromim_t[:,:,col]*alpha
            toim_t[:,:,col] = toim_t[:,:,col]*(1-alpha)
    else:
        alpha = (fromim_t > 0)
        toim_t = fromim_t*alpha + toim_t*(1-alpha)
    
    #plt.imshow(toim_t)
    #plt.show()
    return toim_t,alpha

In [ ]:
def panorama(H,fromim,toim,padding=2400,delta=2400):
    """ 使用单应性矩阵H（使用RANSAC稳健性估计得出），协调两幅图像，创建水平全景图。结果
        为一幅和toim具有相同高度的图像。padding指定填充像素的数目，delta指定额外的平移量""" 
    #print(H)
    #im_plot(fromim)
    #im_plot(toim)
    #imfrom = fromim
    #imto = toim
    
    #检查图像是灰度图像，还是彩色图像
    is_color = len(fromim.shape) == 3
    #print(is_color)
    
    # 用于geometric_transform()的单应性变换
    def transf(p):
        p2 = np.dot(H,[p[0],p[1],1])
        return (p2[0]/p2[2],p2[1]/p2[2])
    
    if H[1,2]<0: # fromim在右边
        print ('warp - right')
        # 变换fromim
        if is_color:
            # 在目标图像的右边填充0
            #im_plot(toim)
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding,3))))
            #im_plot(toim_t)
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],transf,(toim.shape[0],toim.shape[1]+padding))
        else:
            # 在目标图像的右边填充0
            toim_t = np.hstack((toim,np.zeros((toim.shape[0],padding))))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding)) 
    else:
        print ('warp - left')
        #为了补偿填充效果，在左边加入平移量
        H_delta = np.array([[1,0,0],[0,1,-delta],[0,0,1]])
        H = np.dot(H,H_delta)
        #fromim变换
        if is_color:
            # 在目标图像的左边填充0
            toim_t = np.hstack((np.zeros((toim.shape[0],padding,3)),toim))
            fromim_t = np.zeros((toim.shape[0],toim.shape[1]+padding,toim.shape[2]))
            #im_plot(fromim_t)
            for col in range(3):
                fromim_t[:,:,col] = ndimage.geometric_transform(fromim[:,:,col],transf,(toim.shape[0],toim.shape[1]+padding))
            #im_plot(fromim_t)
        else:
            # 在目标图像的左边填充0
            toim_t = np.hstack((np.zeros((toim.shape[0],padding)),toim))
            fromim_t = ndimage.geometric_transform(fromim,transf,(toim.shape[0],toim.shape[1]+padding))
    
    # 协调后返回（将fromim放在toim上）
    #im_plot(fromim_t)
    #im_plot(toim_t)
    if is_color:
        # 所有非黑像素
        alpha = ((fromim_t[:,:,0] * fromim_t[:,:,1] * fromim_t[:,:,2] ) > 0)
        #return fromim_t,toim_t,alpha
        for col in range(3):
            toim_t[:,:,col] = fromim_t[:,:,col]*alpha + toim_t[:,:,col]*(1-alpha)
            
    else:
        alpha = (fromim_t > 0)
        toim_t = fromim_t*alpha + toim_t*(1-alpha)
    
    #plt.imshow(toim_t)
    #plt.show()
    return toim_t